In [377]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Model

# Load data from CSV file -> got this dataset from kaggle
data = pd.read_csv('/Users/prajwalhegde/Downloads/ner_dataset.csv', encoding="latin1")
data.dtypes

Sentence #    object
Word          object
POS           object
Tag           object
dtype: object

In [378]:
curr_sentence = 1
# Iterate through the "Word" column using .iteritems()
for index, sentence_num in data['Sentence #'].iteritems():
    #print(f"Index: {index}, Word: {sentence_num}")
    if (pd.isna(sentence_num)):
        data.loc[index, 'Sentence #'] = curr_sentence
    else:
        curr_sentence = int(sentence_num.split(': ')[1])
        data.loc[index, 'Sentence #'] = curr_sentence
#df['Sentence #'] = df['Sentence #'].astype('int64')

/var/folders/8x/9m1j4l7s593g3365mw4qsq2r0000gn/T/ipykernel_46114/492576798.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, sentence_num in data['Sentence #'].iteritems():


In [379]:
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,47959,they,PRP,O
1048571,47959,responded,VBD,O
1048572,47959,to,TO,O
1048573,47959,the,DT,O
1048574,47959,attack,NN,O


In [380]:
data['Sentence #'] = data['Sentence #'].astype('int64')
data.dtypes

Sentence #     int64
Word          object
POS           object
Tag           object
dtype: object

In [381]:
tag_encoder = LabelEncoder()
data['Tag_index'] = tag_encoder.fit_transform(data['Tag'])

# Group data by sentence and create sequences of words and tags
grouped_data = data.groupby('Sentence #')[['Word', 'Tag_index']].apply(lambda x: x.values.tolist()).to_dict()
grouped_data[1]

[['Thousands', 16],
 ['of', 16],
 ['demonstrators', 16],
 ['have', 16],
 ['marched', 16],
 ['through', 16],
 ['London', 2],
 ['to', 16],
 ['protest', 16],
 ['the', 16],
 ['war', 16],
 ['in', 16],
 ['Iraq', 2],
 ['and', 16],
 ['demand', 16],
 ['the', 16],
 ['withdrawal', 16],
 ['of', 16],
 ['British', 3],
 ['troops', 16],
 ['from', 16],
 ['that', 16],
 ['country', 16],
 ['.', 16]]

In [382]:
# Convert grouped data into sequences of sentences and labels
sentences = []
labels = []
for sentence, group in grouped_data.items():
    words = [item[0] for item in group]
    tags = [item[1] for item in group]
    sentences.append(words)
    labels.append(tags)


In [383]:
word_to_index = {}
for sentence in sentences:
    for word in sentence:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
word_to_index['UNK'] = len(word_to_index)

In [384]:
index_to_tag = {idx: tag for idx, tag in enumerate(tag_encoder.classes_)}
numerical_index = 2
tag_label = index_to_tag[numerical_index]
print(tag_label)  # This will print the original tag label associated with index 1

B-geo


In [385]:
sentences_indices = []

for sentence in sentences:
    sentence_indices = [word_to_index.get(word, word_to_index['UNK']) for word in sentence]
    sentences_indices.append(sentence_indices)
print (sentences_indices[1000])

[61, 1586, 3938, 2679, 151, 846, 4648, 68, 3006, 11, 4649, 21]


In [386]:
num_classes = len(index_to_tag)


# Find the maximum sequence length for both X and y
max_sequence_length = max(max(len(sequence) for sequence in sentences_indices), max(len(sequence) for sequence in labels))

# Pad the input sequences (X) to match the maximum sequence length
X = pad_sequences(sentences_indices, maxlen=max_sequence_length, padding='post', value=0)  # Assuming '0' is the padding value

# Pad the target sequences (y) to match the maximum sequence length
y = pad_sequences(labels, maxlen=max_sequence_length, padding='post', value=0)  # Assuming '0' is the padding value


In [387]:
print(X[0])
print(y[0])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  9 15  1 16 17 18 19 20 21
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[16 16 16 16 16 16  2 16 16 16 16 16  2 16 16 16 16 16  3 16 16 16 16 16
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [388]:
y = np.array(y)

In [389]:
print(X[0])
print(y[0])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  9 15  1 16 17 18 19 20 21
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[16 16 16 16 16 16  2 16 16 16 16 16  2 16 16 16 16 16  3 16 16 16 16 16
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [390]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [391]:
print (max_sequence_length)
print(len(y[0]))
print(len(X[0]))

104
104
104


In [392]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming you have already prepared X_train, X_test, y_train, y_test as described earlier

# Define hyperparameters
embedding_dim = 100  # Choose an appropriate value based on your data
lstm_units = 64     # Number of LSTM units
num_epochs = 2     # Number of training epochs
batch_size = 32     # Batch size
num_tags = 17       # Number of unique tag classes

# Build the bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test))


Epoch 1/2
1199/1199 [==============================] - 57s 46ms/step - loss: 0.1140 - accuracy: 0.9746 - val_loss: 0.0313 - val_accuracy: 0.9913
Epoch 2/2
1199/1199 [==============================] - 56s 46ms/step - loss: 0.0231 - accuracy: 0.9933 - val_loss: 0.0238 - val_accuracy: 0.9928


In [393]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss: {} ".format(results[0]))
print("test accuracy: {} ".format(results[1]))

Evaluate on test data
75/75 [==============================] - 2s 32ms/step - loss: 0.0238 - accuracy: 0.9928
test loss: 0.02381025068461895 
test accuracy: 0.9928426146507263 


In [394]:
print (X_test[0])
print (y_test[0])

predictions = model.predict(X_test)

print (predictions.shape)


[  345   163    93     9   703     1   110    68  6635  7532    93  1165
  9225  9657 17042    93   172   363   496    20  1461  1859  3584  3585
     7  7665  1848   161   898    21     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[16  7 16 16 16 16  2 16  2 10 16  5 13 13 13 16 16  7 16 16 16 16 16 16
 16 16 16 16 16 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
300/300 [==============================] - 4s 11

In [413]:
# Assuming 'model' is your compiled Keras model

# Example sentence
example_sentence = "Millions of demonstrators gathered in Times Square for a cold American beverage."

print (word_to_index['UNK'])

# Tokenize the sentence and convert to word indices
words = example_sentence.split()
words2 = example_sentence2.split()
word_indices = [word_to_index.get(word, word_to_index['UNK']) for word in words]
# Pad the sequence
expected_sequence_length = 104
padded_sequence = pad_sequences([word_indices], maxlen=expected_sequence_length, padding='post', value=0)

print (padded_sequence[0])



# Make predictions
novel_predictions = model.predict(padded_sequence)

# Print predictions
ex_tags = []
print("Predictions for the example sentence:")
for prob in novel_predictions[0]:
    ex_tags.append(index_to_tag[np.argmax(prob)])
print (ex_tags)
    


35178
[ 2502     1     2  3569    11  3630 12371   223    45  1005   329 35178
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
1/1 [==============================] - 0s 16ms/step
Predictions for the example sentence:
['O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'B-gpe', 'I-org', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-a